# APIs (Application Programming Interfaces)

Typical way of getting data (usually JSON or XML)

## Think Communication

Application to another application:
* Send request (with some info/data)
* Get response
    + data
    + service


Examples include:

- Financial transactions
- Posting to Twitter
- Controlling IOT

Always a software-to-software interaction

## Parts on an API

* **Access Permissions**
    + User allowed to ask?
* **API Call/Request**
    + Code used to make API call to implement complicated tasks/features
    + _Methods_: what questions can we ask?
    + _Parameters_: more info to be sent
* **Repsonse**
    + Result of request

## API Types

### Web

Interface can work at both server or client end

Examples:
* [Twitter REST API](https://developer.twitter.com/en/docs.html)
* [Amazon S3 REST API](https://docs.aws.amazon.com/AmazonS3/latest/API/Welcome.html)

### Databases

* Pass back data in specific format
* Generalizable

### Operating Systems

### Hardware

# Client Server Model

Distributed architecture:
1. Servers: provide a service
    - Usually physically separate from clinet
2. Clients: request a service
    - hardware or software

## Client

**Web Example**

Defines what user sees, so it must:
  + Define the page visuals
  + Methods for responding to user interactions

## Server

**Web Example**

Listens to requests (through HTTP):
1. Authenticates client/user
2. Looks at requests
3. Performs actions/processes needed
4. Sends response to client (possibly with more data)

## Database

Can be called from the server to get data for the client

# HTTP Requests

Specific formatting to make a proper request.

Libraries `urllib` & `urllib2` can be complicated 😕

## Code Example

We can use `requests` library to get web page form data, files, and parameters more easily.

In [ ]:
import requests

In [ ]:
# Getting the response from our request 
resp = requests.get('https://www.google.com')

In [ ]:
type(resp)

In [ ]:
print('Response Code:',resp.status_code)
print('Is it OK?',resp.status_code == requests.codes.ok)

### Parts of the response

In [ ]:
# Full HTML doc
print(resp.text)

In [ ]:
display(resp.headers)
# Convert to a dicitonary from `requests.structures.CaseInsensitiveDict`
headers = dict(resp.headers)

In [ ]:
print(headers['Date'])   # Date the response was sent
print(headers['Server']) # Server type (google web service - GWS)

### Passing Parameters

In [ ]:
credentials = {'user_name': 'luigi', 'password': 'i<3peach'}  
r = requests.get('http://httpbin.org/get', params=credentials)

# Note we can only do this since the r.text() is JSON format
results = r.json()
# Don't want my IP floating around 😉
results['origin'] = None

print(r.url)  
display(results)


### HTTP Post

Allows multiples be sent at once 

In [ ]:
# Example won't run without example files
url = 'http://httpbin.org/post'  
file_list = [  
    ('image', ('cat.png', open('cat.png', 'rb'), 'image/png')),
    ('image', ('dog.jpg', open('dog.jpg', 'rb'), 'image/jpg'))
]

r = requests.post(url, files=file_list)  
print(r.text)

# OAuth (Open Authorization)

Avoid abuse by limiting.

Usually we can use personal access tokens (typical for development).
But with large number of authentication, OAuth is most common

1. Get credentials & authorize application (before OAuth)
2. **Authorize** permissions requested
3. **Redirect** use back w/ authorization code
4. **Aquisition** user "recieves" access token

Check out curriculum for full details

## Why use OAuth?

Alternative is essentially a username & password (API key & secret)

Allows access without user password:
**Authentication** separated from **Authorization**

# Example Using An API

## LIFX API

Documentation: https://api.developer.lifx.com/

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
tokens = {
        'lifx' : {
            'token_name': 'Lifx',
            'token': 'c33cf42e79aaf8afc8b647e13b07ff9fe668587c41c722ae6896462f835190ab',
        }
}

In [ ]:
# Specific to today
token = tokens['lifx']['token']

headers = {
    "Authorization": "Bearer %s" % token,
}

response = requests.get('https://api.lifx.com/v1/lights/all', headers=headers)

In [ ]:
lights = response.json()
display(lights)

In [ ]:
pd.DataFrame.from_dict(lights)

In [ ]:
for light in lights:
    print(light['label'])

### Power On

In [ ]:
payload = {
  "states": [
    {
        "selector" : str(lights[1]['id']),
        "power": "on"
    }
  ]
}
response = requests.put('https://api.lifx.com/v1/lights/states', data=json.dumps(payload), headers=headers)
print(response.content)

### Power On w/ Color

In [ ]:
payload = {
  "states": [
    {
        "selector" : str(lights[1]['id']),
            "period": 2,
            "cycles": 5,
            "color": "blue",
            "brightness": 0.5
    }
  ],
  "defaults": {
    "power": "on",
    "saturation": 0,
    "duration": 2.0

}

}

response = requests.put('https://api.lifx.com/v1/lights/states', data=json.dumps(payload), headers=headers)
print(response.content)

### Power Off

In [ ]:
payload = {
  "states": [
    {
        "selector" : str(lights[1]['id']),
        "power": "off"
    }
  ]
}

response = requests.put('https://api.lifx.com/v1/lights/states', data=json.dumps(payload), headers=headers)
print(response.content)